In [1]:
import joblib
import cv2
import numpy as np
import os
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
from ipyfilechooser import FileChooser

# Upload model and image
model_picker = widgets.FileUpload(accept='.pkl', multiple=False, description='Upload Model (.pkl)')
image_picker = widgets.FileUpload(accept='.tif,.jpg,.png', multiple=False, description='Upload Image')

# Output file name and folder picker
output_name = widgets.Text(value='prediction.tif', description='Output Name:')
folder_chooser = FileChooser(os.getcwd())
folder_chooser.title = 'Select Output Folder'

run_button = widgets.Button(description='Run Prediction', button_style='success')
output = widgets.Output()

display(model_picker, image_picker, output_name, folder_chooser, run_button, output)

def on_run_clicked(b):
    with output:
        output.clear_output()
        
        if not model_picker.value or not image_picker.value:
            print("❌ Please upload both model and image.")
            return
        
        if not folder_chooser.selected_path:
            print("❌ Please select an output folder.")
            return

        # Get uploaded files
        model_file = model_picker.value[0]
        image_file = image_picker.value[0]

        # Save them temporarily
        model_path = Path("temp_model.pkl")
        image_path = Path("temp_image.tif")
        model_path.write_bytes(model_file['content'])
        image_path.write_bytes(image_file['content'])

        # Load model and image
        model = joblib.load(model_path)
        new_img = cv2.imread(str(image_path))

        if new_img is None:
            print("❌ Failed to load image.")
            return

        # Predict
        new_features = new_img.reshape(-1, 3)
        predictions = model.predict(new_features)
        pred_img = predictions.reshape(new_img.shape[:2])
        mask = (pred_img * 255).astype('uint8')

        # Save output
        output_file = Path(folder_chooser.selected_path) / output_name.value
        cv2.imwrite(str(output_file), mask)

        print(f"✅ Prediction saved to: {output_file}")
        print(mask)

# Bind run button
run_button.on_click(on_run_clicked)


FileUpload(value=(), accept='.pkl', description='Upload Model (.pkl)')

FileUpload(value=(), accept='.tif,.jpg,.png', description='Upload Image')

Text(value='prediction.tif', description='Output Name:')

FileChooser(path='E:\Faruq\Business\DEEPRESENSE\Experimental\binary classification\water body\Model\XGBoost Al…

Button(button_style='success', description='Run Prediction', style=ButtonStyle())

Output()